In [1]:
!pip install category_encoders

In [2]:
import pandas as pd
import numpy as np
import os
import datetime
import tensorflow as tf
import category_encoders as ce
from sklearn.preprocessing import OneHotEncoder
import math
import pickle

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [5]:
df = pd.read_csv('airbnb.csv')

In [6]:
df.head()

,id,log_price,property_type,room_type,amenities,accommodates,bathrooms,bed_type,cancellation_policy,cleaning_fee,city,description,first_review,host_has_profile_pic,host_identity_verified,host_response_rate,host_since,instant_bookable,last_review,latitude,longitude,name,neighbourhood,number_of_reviews,review_scores_rating,thumbnail_url,zipcode,bedrooms,beds
0,6901257,5.010635,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",3,1.0,Real Bed,strict,True,NYC,"Beautiful, sunlit brownstone 1-bedroom in the ...",6/18/16,t,t,NaN,3/26/12,f,7/18/16,40.696524,-73.991617,Beautiful brownstone 1-bedroom,Brooklyn Heights,2,100.0,https://a0.muscache.com/im/pictures/6d7cbbf7-c...,11201,1.0,1.0
1,6304928,5.129899,Apartment,Entire home/apt,"{""Wireless Internet"",""Air conditioning"",Kitche...",7,1.0,Real Bed,strict,True,NYC,Enjoy travelling during your stay in Manhattan...,8/5/17,t,f,100%,6/19/17,t,9/23/17,40.766115,-73.989040,Superb 3BR Apt Located Near Times Square,Hell's Kitchen,6,93.0,https://a0.muscache.com/im/pictures/348a55fe-4...,10019,3.0,3.0
2,7919400,4.976734,Apartment,Entire home/apt,"{TV,""Cable TV"",""Wireless Internet"",""Air condit...",5,1.0,Real Bed,moderate,True,NYC,The Oasis comes complete with a full backyard ...,4/30/17,t,t,100%,10/25/16,t,9/14/17,40.808110,-73.943756,The Garden Oasis,Harlem,10,92.0,https://a0.muscache.com/im/pictures/6fae5362-9...,10027,1.0,3.0
3,13418779,6.620073,House,Entire home/apt,"{TV,""Cable TV"",Internet,""Wireless Internet"",Ki...",4,1.0,Real Bed,flexible,True,SF,This light-filled home-away-from-home is super...,NaN,t,t,NaN,4/19/15,f,NaN,37.772004,-122.431619,Beautiful Flat in the Heart of SF!,Lower Haight,0,NaN,https://a0.muscache.com/im/pictures/72208dad-9...,94117,2.0,2.0
4,3808709,4.744932,Apartment,Entire home/apt,"{TV,Internet,""Wireless Internet"",""Air conditio...",2,1.0,Real Bed,moderate,True,DC,"Cool, cozy, and comfortable studio located in ...",5/12/15,t,t,100%,3/1/15,t,1/22/17,38.925627,-77.034596,Great studio in midtown DC,Columbia Heights,4,40.0,NaN,20009,0.0,1.0


In [7]:
def wrangle(X):

  X = X.filter(['log_price', 'room_type', 'accommodates', 'bathrooms', 'latitude',
                'longitude','review_scores_rating', 'bedrooms', 'beds', 'TV', 
                'zipcode', 'description'], axis=1)

  X["zipcode"] = X["zipcode"].replace("Near 91304", 91304)
  X["zipcode"] = X["zipcode"].replace("1m", 10023)
  X["zipcode"] = pd.to_numeric(X["zipcode"], errors="coerce")
  X["zipcode"] = X["zipcode"].replace(np.nan, X["zipcode"].median())
  X["zipcode"] = X["zipcode"].astype(str)
  X['zipcode'] = X['zipcode'][:5] 
  X["zipcode"] = X["zipcode"].replace(".", "")  
  X["zipcode"] = X["zipcode"].astype(float)

  X = X.fillna(0)

  encoder = ce.OrdinalEncoder()
  X = encoder.fit_transform(X)

  X = X.astype(float)

  return X

In [8]:
df = wrangle(df)

In [11]:
df['description'] = df['description'].astype(str)

In [12]:
import spacy
import re

nlp = spacy.load('en_core_web_sm') 

stop_words = nlp.Defaults.stop_words.union(['place', 'airbnb', 'travel'])

def tokenize(doc):
    
    lemmas = []
    
    
    
    doc = doc.lower()
    doc = re.sub('[^a-zA-Z 0-9]', '', doc)
    doc = nlp(doc)
    
    for token in doc:
        conditions = (token.text.lower() not in stop_words) and (token.is_punct == False) and (token.pos_ != 'PRON') 
        if conditions:
            lemmas.append(token.lemma_)
    
    return lemmas

In [13]:
df['tokens'] = df['description'].apply(tokenize)

## How does your description compare to those of other homes?

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [46]:
tfidf = TfidfVectorizer(stop_words=stop_words,  
                        ngram_range=(1,2),
                        max_features = 8000,
                        tokenizer=tokenize)

In [16]:
tfidf.fit(df['description'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"
/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['d', 'm', 'nt', 's', 've'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=8000,
                min_df=1, ngram_range=(1, 2), norm='l2', preprocessor=None,
                smooth_idf=True,
                stop_words={"'d", "'ll", "'m", "'re", "'s", "'ve", 'a', 'about',
                            'above', 'across', 'after', 'afterwards', 'again',
                            'against', 'airbnb', 'all', 'almost', 'alone',
                            'along', 'already', 'also', 'although', 'always',
                            'am', 'among', 'amongst', 'amount', 'an', 'and',
                            'another', ...},
                strip_accents=None, sublinear_tf=False,
                token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=<function tokenize at 0x7f3759b59d90>, use_idf=True,
                vocabulary=None)

In [17]:
dtm = tfidf.transform(df['description'])

In [18]:
dtm = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())

In [19]:
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=10, algorithm='kd_tree')

knn.fit(dtm)

NearestNeighbors(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                 radius=1.0)

In [57]:
desc2 = ["""
        Great place, great atmosphere, great kids, love the dogs and food and 
        fun and the room was beautiful and the bed was so comfortable.
        """]

In [22]:
new = tfidf.transform(desc)

In [23]:
knn.kneighbors(new.todense())

(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 array([[  570, 74110, 37053, 18524,   280,  1150,  4626,    61,   134,
          9259]]))

In [49]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline

tfidf = TfidfVectorizer()
lr = LinearRegression()

pipe = Pipeline([('tfidf', tfidf), ('lr', lr)])

In [25]:
df['review_scores_rating'].dtype

dtype('float64')

In [50]:
from sklearn.model_selection import GridSearchCV


parameters = {
    'tfidf__max_features': (500, 1000)
}

grid_search = GridSearchCV(pipe, parameters, cv=2, n_jobs=-1, verbose=9)

grid_search.fit(df['description'], df['review_scores_rating'])

Fitting 2 folds for each of 2 candidates, totalling 4 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   4 | elapsed:    1.9s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed:    2.5s finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('tfidf',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                        n

In [58]:
print(grid_search.predict(desc2))

[72.84319894]
